In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense,CuDNNLSTM, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from janome.tokenizer import Tokenizer
import numpy as np
import random
import glob
import sys
import io
import re
import os

t = Tokenizer()

Using TensorFlow backend.


In [10]:
char_indices = {}  # 辞書初期化
indices_char = {}  # 逆引き辞書初期化
texts= []
chars = []
parts = []
paths = glob.glob('C:/Users/hata/sozoenshu/data/akutagawa/*.txt')
for path in paths:
    word = []
    part = []
    print('reading ' , path)
    binarydata = open(path,'rb').read()
    text = binarydata.decode('shift_jis')
    text = re.split(r'\-{5,}',text)[2]
    text = re.split(r'底本：',text)[0]
    text = text.replace('|','')
    text = text.replace('\u3000','')
    text = re.sub(r'《.+?》','',text) # ルビをとる
    text = re.sub(r'※','',text) # ※をとる
    text = re.sub(r'sentimentalisme','センチメンタリズム',text)
    text = re.sub(r'\r\n','',text)
    text = re.sub(r'―','',text)
    text = re.sub(r'［＃.+?］','',text) # 入力注をとる
    texts.append(text)
    print('corpus length:', len(text))
    malist = t.tokenize(text)
    for w in malist:
        word.append(w.surface)
        part.append(w.part_of_speech.split(','))
    chars.append(word)
    parts.append(part)

POS = []
for i in range(len(parts)):
    for j in range(len(parts[i])):
        if parts[i][j] not in POS:
            POS.append(parts[i][j])

print('Part of Speech:',POS)
print('Part of Speech length:', len(POS))
print('texts length:',len(texts))
print('chars length:', len(chars))

# 辞書の作成
global_chars = []
for i in range(len(chars)):
    print('chars['+str(i)+']:', len(chars[i]))
    for word in chars[i]:
        if not word in global_chars:  # 未登録なら
            global_chars.append(word)  # 登録する

char_indices = dict((c,i) for i,c in enumerate(global_chars))
indices_char = dict((i,c) for i,c in enumerate(global_chars))

POS_indices = dict((str(c),i) for i,c in enumerate(POS))
indices_POS = dict((i,str(c)) for i,c in enumerate(POS))

print('global_chars length', len(global_chars))

reading  C:/Users/hata/sozoenshu/data/akutagawa\ababababa.txt
corpus length: 6206
reading  C:/Users/hata/sozoenshu/data/akutagawa\afterlife_eiyaku_kosei.txt
corpus length: 2817
reading  C:/Users/hata/sozoenshu/data/akutagawa\agunino_kami.txt
corpus length: 7868
reading  C:/Users/hata/sozoenshu/data/akutagawa\ahen.txt
corpus length: 2616
reading  C:/Users/hata/sozoenshu/data/akutagawa\aidokushono_insho.txt
corpus length: 965
reading  C:/Users/hata/sozoenshu/data/akutagawa\aki.txt
corpus length: 10929
reading  C:/Users/hata/sozoenshu/data/akutagawa\akuma.txt
corpus length: 1535
reading  C:/Users/hata/sozoenshu/data/akutagawa\akutagawa_ryunosuke_kashu.txt
corpus length: 2059
reading  C:/Users/hata/sozoenshu/data/akutagawa\anchu_mondo.txt
corpus length: 5788
reading  C:/Users/hata/sozoenshu/data/akutagawa\anikino_yona_kokoromochi.txt
corpus length: 1020
reading  C:/Users/hata/sozoenshu/data/akutagawa\anokorono_jibun.txt
corpus length: 15660
reading  C:/Users/hata/sozoenshu/data/akutagawa\a

reading  C:/Users/hata/sozoenshu/data/akutagawa\honyaku_shohin.txt
corpus length: 1109
reading  C:/Users/hata/sozoenshu/data/akutagawa\hoonki.txt
corpus length: 14532
reading  C:/Users/hata/sozoenshu/data/akutagawa\hyottoko.txt
corpus length: 5702
reading  C:/Users/hata/sozoenshu/data/akutagawa\ichiban_kinorino_surutoki.txt
corpus length: 1782
reading  C:/Users/hata/sozoenshu/data/akutagawa\iida_dakotsu.txt
corpus length: 1465
reading  C:/Users/hata/sozoenshu/data/akutagawa\imogayu.txt
corpus length: 14395
reading  C:/Users/hata/sozoenshu/data/akutagawa\inuto_fue.txt
corpus length: 7648
reading  C:/Users/hata/sozoenshu/data/akutagawa\isho.txt
corpus length: 1949
reading  C:/Users/hata/sozoenshu/data/akutagawa\ism_to_yu_gono.txt
corpus length: 826
reading  C:/Users/hata/sozoenshu/data/akutagawa\issekiwa.txt
corpus length: 5820
reading  C:/Users/hata/sozoenshu/data/akutagawa\itojo_oboegaki.txt
corpus length: 6763
reading  C:/Users/hata/sozoenshu/data/akutagawa\itokara.txt
corpus length: 

reading  C:/Users/hata/sozoenshu/data/akutagawa\myona_hanashi.txt
corpus length: 4750
reading  C:/Users/hata/sozoenshu/data/akutagawa\nagasaki.txt
corpus length: 302
reading  C:/Users/hata/sozoenshu/data/akutagawa\nagasaki_shohin.txt
corpus length: 2471
reading  C:/Users/hata/sozoenshu/data/akutagawa\nankinno_kirisuto.txt
corpus length: 9821
reading  C:/Users/hata/sozoenshu/data/akutagawa\natsumesenseito_takitasan.txt
corpus length: 641
reading  C:/Users/hata/sozoenshu/data/akutagawa\negi.txt
corpus length: 8034
reading  C:/Users/hata/sozoenshu/data/akutagawa\nenmatsuno_ichinichi.txt
corpus length: 2947
reading  C:/Users/hata/sozoenshu/data/akutagawa\nenniwa_nen'o_irecho.txt
corpus length: 816
reading  C:/Users/hata/sozoenshu/data/akutagawa\nezumikozo_jirokichi.txt
corpus length: 12233
reading  C:/Users/hata/sozoenshu/data/akutagawa\nidai.txt
corpus length: 2133
reading  C:/Users/hata/sozoenshu/data/akutagawa\nijunengono_senso.txt
corpus length: 4486
reading  C:/Users/hata/sozoenshu/da

reading  C:/Users/hata/sozoenshu/data/akutagawa\sobyo_sandai.txt
corpus length: 2412
reading  C:/Users/hata/sozoenshu/data/akutagawa\sogiki.txt
corpus length: 3968
reading  C:/Users/hata/sozoenshu/data/akutagawa\sonokorono_akamonseikatsu.txt
corpus length: 1191
reading  C:/Users/hata/sozoenshu/data/akutagawa\sosaku.txt
corpus length: 2352
reading  C:/Users/hata/sozoenshu/data/akutagawa\soseki_sanbono_aki.txt
corpus length: 1610
reading  C:/Users/hata/sozoenshu/data/akutagawa\soseki_sanbono_fuyu.txt
corpus length: 1531
reading  C:/Users/hata/sozoenshu/data/akutagawa\soshun.txt
corpus length: 3069
reading  C:/Users/hata/sozoenshu/data/akutagawa\susanono_mikoto.txt
corpus length: 42370
reading  C:/Users/hata/sozoenshu/data/akutagawa\sutego.txt
corpus length: 4662
reading  C:/Users/hata/sozoenshu/data/akutagawa\tabakoto_akuma.txt
corpus length: 6100
reading  C:/Users/hata/sozoenshu/data/akutagawa\tabatajin.txt
corpus length: 1753
reading  C:/Users/hata/sozoenshu/data/akutagawa\tabata_nikki

chars[54]: 2545
chars[55]: 1378
chars[56]: 3995
chars[57]: 1165
chars[58]: 723
chars[59]: 3719
chars[60]: 7639
chars[61]: 3788
chars[62]: 1713
chars[63]: 2512
chars[64]: 8246
chars[65]: 13864
chars[66]: 13247
chars[67]: 8015
chars[68]: 919
chars[69]: 303
chars[70]: 4649
chars[71]: 16512
chars[72]: 5608
chars[73]: 665
chars[74]: 765
chars[75]: 3952
chars[76]: 4901
chars[77]: 6131
chars[78]: 535
chars[79]: 1599
chars[80]: 752
chars[81]: 12510
chars[82]: 1967
chars[83]: 7418
chars[84]: 886
chars[85]: 110
chars[86]: 6466
chars[87]: 1122
chars[88]: 448
chars[89]: 1131
chars[90]: 953
chars[91]: 6977
chars[92]: 14686
chars[93]: 13961
chars[94]: 2032
chars[95]: 805
chars[96]: 9598
chars[97]: 3859
chars[98]: 1162
chars[99]: 1003
chars[100]: 10460
chars[101]: 5202
chars[102]: 1352
chars[103]: 554
chars[104]: 3866
chars[105]: 4713
chars[106]: 464
chars[107]: 394
chars[108]: 26358
chars[109]: 18198
chars[110]: 18444
chars[111]: 390
chars[112]: 383
chars[113]: 1264
chars[114]: 1149
chars[115]: 5479

In [11]:
text

"一放屁アンドレエフに百姓が鼻糞をほじる描写がある。フランスに婆さんが小便をする描写がある。しかし屁をする描写のある小説にはまだ一度も出あつたことはない。出あつたことのないといふのは、西洋の小説にはと云ふ意味である。日本の小説にはない訣ではない。その一つは青木健作氏の何とかいふ女工の小説である。駈落ちをした女工が二人、干藁か何かの中に野宿する。夜明に二人とも目がさめる。一人がぷうとおならをする。もう一人がくすくす笑ひ出すたしかそんな筋だつたと思ふ。その女工の屁をする描写は予の記憶に誤りがなければ、甚だ上品に出来上つてゐた。予は此の一段を読んだ為に、今日もなほ青木氏の手腕に敬意を感じてゐる位なものである。もう一つは中戸川吉二氏の何とか云ふ不良少年の小説である。これはつい三四箇月以前、サンデイ毎日に出てゐたのだから、知つてゐる読者も多いかも知れない。不良少年に口説かれた女が際どい瞬間におならをする、その為に折角醸されたエロチツクな空気が消滅する、女は妙につんとしてしまふ、不良少年も手が出せなくなる大体かう云ふ小説だつた。この小説も巧みに書きこなしてある。青木氏の小説に出て来る女工は必しもおならをしないでも好い。しかし中戸川氏の小説に出て来る女は嫌でもおならをする必要がある。しなければ成り立たない。だから屁は中戸川氏を得た後始めて或重大な役目を勤めるやうになつたと云ふべきである。しかしこれは近世のことである。宇治拾遺物語によれば、藤大納言忠家も、「いまだ殿上人におはしける時、びびしき色好みなりける女房ともの云ひて、夜更くるほどに月は昼よりもあかかりけるに」たへ兼ねてひき寄せたら、女は「あなあさまし」と云ふ拍子に大きいおならを一つした。忠家はこの屁を聞いた時に「心うきことにも逢ひぬるかな。世にありて何かはせん。出家せん」と思ひ立つた。けれども、つらつら考へて見れば、何も女が屁をしたからと云つて、坊主にまでなるには当りさうもない。忠家は其処に気がついたから、出家することだけは見合せたが、その場は逃げ出したさうである。すると中戸川氏の小説も文学史的に批評すれば、前人未発と云ふことは出来ない。しかし断えたるを継いだ功は当然同氏に属すべきである。この功は多分中戸川氏自身の予想しなかつたところであらう。しかし功には違ひないから、序に此処に吹聴することにした。二女と影紋服を着た西洋人は

In [12]:
del text
del texts
del paths


# 学習データを作る
maxlen = 8
step = 4
X = []
Y = []
sentences = []
next_chars = []
pos = []
next_pos = []
for j in range(len(chars)):
    for i in range(0, len(chars[j]) - maxlen, step):
        sentences.append(chars[j][i: i + maxlen])
        next_chars.append(chars[j][i + maxlen])
        pos.append(parts[j][i: i + maxlen])
        next_pos.append(parts[j][i + maxlen])
    print('nb sequences:', len(sentences))
    print('Vectorization...')
    x = np.zeros((len(sentences), maxlen, len(global_chars)+len(POS)), dtype=np.bool)
    y = np.zeros((len(sentences),len(global_chars)+len(POS)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence): #sentence = sentences[i], char = sentence[t]
            x[i, t, char_indices[char]] = 1
            x[i, t, len(global_chars)+POS_indices[str(pos[i][t])]] = 1
        y[i, char_indices[next_chars[i]]] = 1
        y[i, len(global_chars)+POS_indices[str(pos[i][t])]] = 1
    X.append(x)
    Y.append(y)

nb sequences: 1050
Vectorization...
nb sequences: 1344
Vectorization...
nb sequences: 2607
Vectorization...
nb sequences: 3071
Vectorization...
nb sequences: 3228
Vectorization...
nb sequences: 5146
Vectorization...
nb sequences: 5418
Vectorization...
nb sequences: 5752
Vectorization...
nb sequences: 6753
Vectorization...
nb sequences: 6920
Vectorization...
nb sequences: 9585
Vectorization...
nb sequences: 10270
Vectorization...


MemoryError: 

In [70]:
# build the model: a single LSTM
def loss(y, pred):
    s = pred-y
    s = np.sum(s)
    return (abs(s)*10)**2

print('Build model...')
learning_rate = 0.01
model = Sequential()

model.add(CuDNNLSTM(256, input_shape=(maxlen, len(global_chars)+len(POS))))
model.add(BatchNormalization())
model.add(Dropout(0.15))

model.add(Dense(512, activation='relu'))

model.add(Dense((len(global_chars)+len(POS)), activation='softmax'))

print(model.summary())

optimizer = 'Adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer) 

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 512)               24233984  
_________________________________________________________________
batch_normalization_8 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_9 (Dense)              (None, 11320)             5807160   
Total params: 30,305,848
Trainable params: 30,304,824
Non-trainable params: 1,024
_________________________________________________________________
None


In [83]:
def sample(preds, temperature=1.0):
    preds = preds[:len(global_chars)]
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    #char_preds = preds[:len(global_chars)]
    #pos_preds = preds[len(global_chars):]
    #return np.argmax(char_preds), np.argmax(pos_preds)
    return np.argmax(probas)

for epochs in range(100):
    for number_of_dataset in range(len(X)):
        for iteration in range(1,3):
            print('\n')
            print('-' *50)
            print('epochs: ', epochs+1)
            print('number_of_dataset: ', number_of_dataset)
            print('繰り返し回数: ', iteration)
            model.fit(X[number_of_dataset], Y[number_of_dataset], batch_size=2048, shuffle = True, epochs=1)

            #start_index = random.randint(0, len(chars[number_of_dataset])-maxlen-1)
            start_index = 0
            
            for diversity in [1.2]:
                print()
                print('-----diveristy', diversity)

                generated = ''
                sentence_chars = chars[number_of_dataset][start_index: start_index + maxlen]
                sentence_parts = parts[number_of_dataset][start_index: start_index + maxlen]
                sentence = ''.join(sentence_chars)
                generated += sentence
                print('----- Seedを生成しました: "' + sentence + '"')
                sys.stdout.write(generated)

                for i in range(10):
                    x = np.zeros((len(sentences), maxlen, len(global_chars)+len(POS)), dtype=np.bool)
                    for t, char in enumerate(sentence_chars):
                        x[0, t, char_indices[char]] = 1.
                        x[0, t, POS_indices[str(sentence_parts[t])]] = 1.

                    preds = model.predict(x, verbose=0)[0]
                    next_index = sample(preds, diversity)
                    next_char = indices_char[next_index]
                    generated += next_char
                    sentence_chars = sentence_chars[1:]
                    sentence_chars.append(next_char)
                    
                    sys.stdout.write(next_char)
                    sys.stdout.flush()

--------------------------------------------------
繰り返し回数:  1
Epoch 1/1
1081/1081 [==============================] - 1s 1ms/step - loss: 1.5878

-----diveristy 0.2
----- Seedを生成しました: "たり、せわしないことです。"
たり、せわしないことです。ますますますますますますますますますます

C:\Users\hata\Anaconda3\envs\kaken\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


ますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますます
-----diveristy 0.5
----- Seedを生成しました: "たり、せわしないことです。"
たり、せわしないことです。ますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますます
-----diveristy 1.0
----- Seedを生成しました: "たり、せわしないことです。"
たり、せわしないことです。ますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますます
-----diveristy 1.2
----- Seedを生成しました: "たり、せわしないことです。"
たり、せわしないことです。ますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますますます----------------

KeyboardInterrupt: 

In [6]:
a=['a','b','c']
a=a[1:]
a.append('de')
print(a)

['b', 'c', 'de']


In [ ]:
text = s
text =Tokenizer().tokenize(text, wakati=True)  # 分かち書きする
chars = text
matasaburo = text

for word in chars:
    if not word in char_indices:  # 未登録なら
        char_indices[word] = count  # 登録する      
        count +=1
        print(count,word)  # 登録した単語を表示
        
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 8
step = 1
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x0 = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y0 = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x0[i, t, char_indices[char]] = 1
    y0[i, char_indices[next_chars[i]]] = 1

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)    

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2]:  # diversity 
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        # sentence はリストなので文字列へ変換して使用
        generated += "".join(sentence)
        print(sentence)
        
        # sentence はリストなので文字列へ変換して使用
        print('----- Generating with seed: "' + "".join(sentence)+ '"')
        sys.stdout.write(generated)


        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:]
            # sentence はリストなので append で結合する
            sentence.append(next_char)  

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])